**Momento de Retroalimentación: Módulo 1 Utilización, procesamiento y visualización de grandes volúmenes de datos (Portafolio Análisis)**

Daniel Flores Rodíguez - A01734184

Lo que se realiza como primera instacia es importar todas las herramientas que tenemos que instalar para poder intalar pyspark y tambien poder inicializarlo.

In [1]:
#Bibliotecas de Pyspark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2//spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [2]:
# Configuramos Spark con Python
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 52.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=0934dbe7a32afa1bcc8e0fab80e32afbe41b8f1bc3e9604a52bb219824dde49e
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
# Establecemos las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# Inicializamos la instalacion de Spark
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.2-bin-hadoop3.2'

Posteriormente de realizar las instalaciones y las iniciaziones pertienentes, se realiza la configuracion de nuestro Drive, para poder acceder a los datos con lo que se va a trabajar.

In [4]:
from google.colab import drive

drive.mount("/content/gdrive")

%cd "/content/gdrive/Shareddrives/Reto AI SEPTIMO"

Mounted at /content/gdrive
/content/gdrive/Shareddrives/Reto AI SEPTIMO


En este caso, verificamos que la instalacion de PySpark sea correcta y que sea funcional para poder trabajar posteriormente sin ningun problema.

In [5]:
#Verificar la funcionalidad de Pyspark 

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,FloatType 


spark_session = SparkSession.builder.appName('Modulo1_Datos_Grandes').config("spark.driver.memory", "16g").getOrCreate()
spark_session

Sabiendo que se utlizara un archivo de datos el cual ya está limpio, pues se utilizó como información para la realziacion de un analisis en nuestro bloque anterior, es por eso que unicamente leeremos el archivo con nuestra sesion de spark 'spark session', el archivo que leeremos es un csv.

In [8]:
df_spark = spark_session.read.csv('telcom_numeric.csv')
df_spark

# podemos asegurarnos de que no haya datos NaN, a traves del siguiente proceso, sin emabrgo, como se mencionó antiormente, el archivo deberia estar limpio para poder trabajar con este. 
df_spark = df_spark.na.drop("all")

In [9]:
df_spark.show()

+----+------+-----------+------------+------------------+------------------+--------------------+------------------+-------------+--------------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------------+-------------+-------------+----------+------------+--------------------+---------------+------+--------------------+-----------------+
| _c0|   _c1|        _c2|         _c3|               _c4|               _c5|                 _c6|               _c7|          _c8|           _c9|       _c10|              _c11|                _c12|                _c13|                _c14|                _c15|              _c16|                _c17|             _c18|         _c19|         _c20|      _c21|        _c22|                _c23|           _c24|  _c25|                _c26|             _c27|
+----+------+-----------+------------+------------------+------------------+----------------

Tal y como se puede observar y tras haber visualizado los datos a traves de la funcion show(), observamos un problema. Basicamente la primera fila (el cual deberia ser el encabezado) se cuenta como una simple serie de registros, es decir como una serie de datos del mismo dataframe, cuando deberian ser el encabezado, se procedera a arreglar este problema.

Para poder arreglar esta situacion que se menciona, volveremos a leer el archyivo csv, pero con la diferencia de que vamos a agregar una funcion para que podamos tomar el inicio del dataframe como el encabezado.

Para eso utilizaremos option('header','true')

In [13]:
data_spark_header = spark_session.read.option('header','true').csv('telcom_numeric.csv')
data_spark_header


DataFrame[_c0: string, TARGET: string, YEAR_JOINED: string, CURRENT_YEAR: string, BILL_AMOUNT: string, PAID_AMOUNT: string, PAYMENT_TRANSACTIONS: string, PARTY_REV: string, PREPAID_LINES: string, POSTPAID_LINES: string, OTHER_LINES: string, LINE_REV: string, MOUS_TO_LOCAL_MOBILES: string, MOUS_FROM_LOCAL_MOBILES: string, MOUS_TO_LOCAL_LANDLINES: string, MOUS_FROM_LOCAL_LANDLINES: string, MOUS_TO_INT_NUMBER: string, MOUS_FROM_INT_NUMBER: string, DATA_IN_BNDL: string, DATA_OUT_BNDL: string, DATA_USG_PAYG: string, COMPLAINTS: string, Years_stayed: string, PTY_PROFILE_SUB_TYPE: string, PARTY_GENDER_CD: string, STATUS: string, SOCIO_ECONOMIC_SEGMENT: string, PARTY_NATIONALITY: string]

In [14]:
data_spark_header.show()

+---+------+-----------+------------+------------------+------------------+--------------------+------------------+-------------+--------------+-----------+------------------+---------------------+-----------------------+-----------------------+-------------------------+------------------+--------------------+-----------------+-------------+-------------+----------+------------+--------------------+---------------+------+----------------------+-----------------+
|_c0|TARGET|YEAR_JOINED|CURRENT_YEAR|       BILL_AMOUNT|       PAID_AMOUNT|PAYMENT_TRANSACTIONS|         PARTY_REV|PREPAID_LINES|POSTPAID_LINES|OTHER_LINES|          LINE_REV|MOUS_TO_LOCAL_MOBILES|MOUS_FROM_LOCAL_MOBILES|MOUS_TO_LOCAL_LANDLINES|MOUS_FROM_LOCAL_LANDLINES|MOUS_TO_INT_NUMBER|MOUS_FROM_INT_NUMBER|     DATA_IN_BNDL|DATA_OUT_BNDL|DATA_USG_PAYG|COMPLAINTS|Years_stayed|PTY_PROFILE_SUB_TYPE|PARTY_GENDER_CD|STATUS|SOCIO_ECONOMIC_SEGMENT|PARTY_NATIONALITY|
+---+------+-----------+------------+------------------+----------

Como podemos observar, en el anterior dataframe, el problema con el header esta solucionado, sin embargo, este archivo ha sido leido por Spark, de tal manera que los datos son todos de tipo string, por lo que nuevamente tendremos que ajustar esto para poder trabajar y vizualizar los datos. 

Para esto usaremos la funcion de conversión y un ciclo for para poder iterar a traves de todas las columnas y filas de datos.

In [16]:
#podemos realziar la comprobacion de que el actual dataset, esta compuesto de puros datos de tipo string

data_spark_header.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- TARGET: string (nullable = true)
 |-- YEAR_JOINED: string (nullable = true)
 |-- CURRENT_YEAR: string (nullable = true)
 |-- BILL_AMOUNT: string (nullable = true)
 |-- PAID_AMOUNT: string (nullable = true)
 |-- PAYMENT_TRANSACTIONS: string (nullable = true)
 |-- PARTY_REV: string (nullable = true)
 |-- PREPAID_LINES: string (nullable = true)
 |-- POSTPAID_LINES: string (nullable = true)
 |-- OTHER_LINES: string (nullable = true)
 |-- LINE_REV: string (nullable = true)
 |-- MOUS_TO_LOCAL_MOBILES: string (nullable = true)
 |-- MOUS_FROM_LOCAL_MOBILES: string (nullable = true)
 |-- MOUS_TO_LOCAL_LANDLINES: string (nullable = true)
 |-- MOUS_FROM_LOCAL_LANDLINES: string (nullable = true)
 |-- MOUS_TO_INT_NUMBER: string (nullable = true)
 |-- MOUS_FROM_INT_NUMBER: string (nullable = true)
 |-- DATA_IN_BNDL: string (nullable = true)
 |-- DATA_OUT_BNDL: string (nullable = true)
 |-- DATA_USG_PAYG: string (nullable = true)
 |-- COMPLAINTS: string (n

In [17]:
for column in data_spark_header.columns:
  # excluimos aquellos datos que no estan en las columnas de ser convertidos en enteros.
  if column not in ["_c0", "TARGET", "YEAR_JOINED", "CURRENT_YEAR", "PAYMENT_TRANSACTIONS", "PREPAID_LINES", "POSTPAID_LINES", "OTHER_LINES", "COMPLAINTS", "PARTY_GENDER_CD", "STATUS", "SOCIO_ECONOMIC_SEGMENT", "PARTY_NATIONALITY"]:
    data_spark_header = data_spark_header.withColumn(column, col(column).cast(FloatType()))
  else:
    data_spark_header = data_spark_header.withColumn(column, col(column).cast(IntegerType()))


#mostramos el df con la transormación realizada.
data_spark_header.show()

+---+------+-----------+------------+-----------+-----------+--------------------+---------+-------------+--------------+-----------+---------+---------------------+-----------------------+-----------------------+-------------------------+------------------+--------------------+------------+-------------+-------------+----------+------------+--------------------+---------------+------+----------------------+-----------------+
|_c0|TARGET|YEAR_JOINED|CURRENT_YEAR|BILL_AMOUNT|PAID_AMOUNT|PAYMENT_TRANSACTIONS|PARTY_REV|PREPAID_LINES|POSTPAID_LINES|OTHER_LINES| LINE_REV|MOUS_TO_LOCAL_MOBILES|MOUS_FROM_LOCAL_MOBILES|MOUS_TO_LOCAL_LANDLINES|MOUS_FROM_LOCAL_LANDLINES|MOUS_TO_INT_NUMBER|MOUS_FROM_INT_NUMBER|DATA_IN_BNDL|DATA_OUT_BNDL|DATA_USG_PAYG|COMPLAINTS|Years_stayed|PTY_PROFILE_SUB_TYPE|PARTY_GENDER_CD|STATUS|SOCIO_ECONOMIC_SEGMENT|PARTY_NATIONALITY|
+---+------+-----------+------------+-----------+-----------+--------------------+---------+-------------+--------------+-----------+-------

In [18]:
data_spark_header.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- YEAR_JOINED: integer (nullable = true)
 |-- CURRENT_YEAR: integer (nullable = true)
 |-- BILL_AMOUNT: float (nullable = true)
 |-- PAID_AMOUNT: float (nullable = true)
 |-- PAYMENT_TRANSACTIONS: integer (nullable = true)
 |-- PARTY_REV: float (nullable = true)
 |-- PREPAID_LINES: integer (nullable = true)
 |-- POSTPAID_LINES: integer (nullable = true)
 |-- OTHER_LINES: integer (nullable = true)
 |-- LINE_REV: float (nullable = true)
 |-- MOUS_TO_LOCAL_MOBILES: float (nullable = true)
 |-- MOUS_FROM_LOCAL_MOBILES: float (nullable = true)
 |-- MOUS_TO_LOCAL_LANDLINES: float (nullable = true)
 |-- MOUS_FROM_LOCAL_LANDLINES: float (nullable = true)
 |-- MOUS_TO_INT_NUMBER: float (nullable = true)
 |-- MOUS_FROM_INT_NUMBER: float (nullable = true)
 |-- DATA_IN_BNDL: float (nullable = true)
 |-- DATA_OUT_BNDL: float (nullable = true)
 |-- DATA_USG_PAYG: float (nullable = true)
 |-- COMPLAINTS: integer (nulla

Podemos observar el cambio/transformacion realizada en el anteiror esquema de los datos, por lo que ahora podremos empezar a trabajar con los datos correspondientes.

Debido a que para trabajar en Spark necesitamos una estuctura de datos diferente, en este caso seria un datagrame RDD. Para la realización de esto se usará una libreria de Milib y asi poder realziar nuestros modelos y predicciones.

In [19]:
rdd_spark = data_spark_header.rdd.map(lambda x: LabeledPoint(x[1], x[2:]))
rdd_spark.take(5)

[LabeledPoint(0.0, [1994.0,2019.0,931.2089233398438,812.1749877929688,1.0,5968.7001953125,2.0,6.0,2.0,945.0399780273438,1004.0700073242188,35.849998474121094,34.01499938964844,72.07499694824219,141.83999633789062,56.1150016784668,11944.0791015625,0.0,0.0,0.0,25.0,2.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,431.0826110839844,486.5,1.0,6245.1416015625,6.0,4.0,2.0,493.81500244140625,159.0500030517578,10.595000267028809,7.715000152587891,11.75,5.110000133514404,0.0,9903.158203125,0.0,0.0,0.0,25.0,1.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,50.61964416503906,52.814998626708984,1.0,1666.48828125,2.0,3.0,1.0,50.29999923706055,0.0,0.0,0.0,0.0,0.0,0.0,0.1025390625,0.0,0.0,0.0,25.0,2.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,399.71002197265625,422.2349853515625,1.0,2522.00830078125,3.0,4.0,3.0,406.586669921875,288.80499267578125,158.5,2.6700000762939453,15.96500015258789,0.0,0.0,3600.322265625,0.0,0.0,0.0,25.0,1.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.

Ya teniendo listo mi rdd, se procede a realizar la separacion de los datos en train y test, con estos, podremos entrenar un modelo, y así poder realizar predicciones.

In [20]:
# Split the data into training and test sets (30% held out for testing)
traingData, testData = rdd_spark.randomSplit([0.7, 0.3])

Ahora, se procedera a realzar el modelo de clasificación a traves de la funcion SVMWithSGD de la libreria Milib.

In [21]:
# se importan las librerias a usar 
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint


# se realzia el entrenamiento del modelo
modelo = SVMWithSGD.train(traingData, iterations=100)

In [22]:
#mostramos el resultado del modelo

modelo

(weights=[-11.144718392757138,-13.832418832263338,-87.46252288271438,-91.85885717149162,-0.00807569481769168,-27.354546897848447,1.2796112839282325,2.3688828661296872,0.1987832769163532,-89.82270578249818,-140.74887882872636,-41.03020374089381,-6.514077297272993,-9.418164297886428,-0.21189657922962035,-2.1127422075559807,-252.625865689159,-0.0010539469969812194,-56.23094774273536,-0.0014631355782538582,-2.6877004395057136,-0.028653946142214108,0.01617130619468812,0.21586123239310775,0.10902734362923605,-8.422656020000563], intercept=0.0)

Para poder validar este modelo se procede a realziar un calculo del error que hay para poder ver la presicion del modelo generado...

In [25]:
predictsLabels = testData.map(lambda x: (x.label, modelo.predict(x.features)))
errTrain = predictsLabels.filter(lambda pl: pl[0] != pl[1]).count() / float(testData.count())
print("Training Error= "+str(errTrain))

Training Error= 0.053088168713762034
